# data.partitioners

> Implement three FL scnarios. IID, Non-IID, and Extended-non-IID.


In [ ]:
#| default_exp optimizers

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import os
import random
import torch
from fastcore.utils import *

In [ ]:
#| export
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
#| export
import math
from torch import Tensor
from torch.optim.optimizer import Optimizer
from typing import List, Optional

## pFedMe Optimizer

In [ ]:
#| export
class pFedMeOptimizer(Optimizer):
    def __init__(self, params, lr=0.01, lambda_=0.1 , mu = 0.001):
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        defaults = dict(lr=lr, lambda_=lambda_, mu = mu)
        super(pFedMeOptimizer, self).__init__(params, defaults)
    
    def step(self, local_weight_updated, closure=None):
        loss = None
        if closure is not None:
            loss = closure
        for group in self.param_groups:
            for p, localweight in zip( group['params'], local_weight_updated.parameters()):
                p.data = p.data - group['lr'] * (p.grad.data + group['lambda_'] * (p.data - localweight.data) + group['mu']*p.data)
        return  group['params'], loss
    
    def update_param(self, local_weight_updated, closure=None):
        loss = None
        if closure is not None:
            loss = closure
        weight_update = local_weight_updated.copy()
        for group in self.param_groups:
            for p, localweight in zip( group['params'], weight_update):
                p.data = localweight.data
        #return  p.data
        return  group['params']

## Per-FedAvg Optimizer

In [ ]:
#| export
class PerFedAvgOpt(Optimizer):
    def __init__(self, params, lr):
        defaults = dict(lr=lr)
        super(PerFedAvgOpt, self).__init__(params, defaults)

    def step(self, closure=None, beta = 0):
        loss = None
        if closure is not None:
            loss = closure

        for group in self.param_groups:
            # print(group)
            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                if(beta != 0):
                    p.data.add_(-beta, d_p)
                else:     
                    p.data.add_(-group['lr'], d_p)
        return loss

## Sophia

In [ ]:
#| export
def _single_tensor_sophiag(params: List[Tensor],
                         grads: List[Tensor],
                         exp_avgs: List[Tensor],
                         hessian: List[Tensor],
                         state_steps: List[Tensor],
                         *,
                         bs: int,
                         beta1: float,
                         beta2: float,
                         rho: float,
                         lr: float,
                         weight_decay: float,
                         maximize: bool,
                         capturable: bool):

    for i, param in enumerate(params):
        grad = grads[i] if not maximize else -grads[i]
        exp_avg = exp_avgs[i]
        hess = hessian[i]
        step_t = state_steps[i]

        if capturable:
            assert param.is_cuda and step_t.is_cuda and bs.is_cuda 
            
        if torch.is_complex(param):
            grad = torch.view_as_real(grad)
            exp_avg = torch.view_as_real(exp_avg)
            hess = torch.view_as_real(hess)
            param = torch.view_as_real(param)

        # update step
        step_t += 1

        # Perform stepweight decay
        param.mul_(1 - lr * weight_decay)

        # Decay the first and second moment running average coefficient
        exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
        
        if capturable:
            step_size = lr 
            step_size_neg = step_size.neg()

            ratio = (exp_avg.abs() / (rho * bs * hess + 1e-15)).clamp(None,1)
            param.addcmul_(exp_avg.sign(), ratio, value=step_size_neg)
        else:
            step_size_neg = - lr 
            
            ratio = (exp_avg.abs() / (rho * bs * hess + 1e-15)).clamp(None,1)
            param.addcmul_(exp_avg.sign(), ratio, value=step_size_neg)

In [ ]:
#| export
def sophiag(params: List[Tensor],
          grads: List[Tensor],
          exp_avgs: List[Tensor],
          hessian: List[Tensor],
          state_steps: List[Tensor],
          capturable: bool = False,
          *,
          bs: int,
          beta1: float,
          beta2: float,
          rho: float,
          lr: float,
          weight_decay: float,
          maximize: bool):

    if not all(isinstance(t, torch.Tensor) for t in state_steps):
        raise RuntimeError("API has changed, `state_steps` argument must contain a list of singleton tensors")

    
    func = _single_tensor_sophiag

    func(params,
         grads,
         exp_avgs,
         hessian,
         state_steps,
         bs=bs,
         beta1=beta1,
         beta2=beta2,
         rho=rho,
         lr=lr,
         weight_decay=weight_decay,
         maximize=maximize,
         capturable=capturable)

In [ ]:
#| export
class SophiaG(Optimizer):
    def __init__(self, params, lr=1e-4, betas=(0.965, 0.99), rho = 0.04,
         weight_decay=1e-1, *, maximize: bool = False,
         capturable: bool = False):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        if not 0.0 <= rho:
            raise ValueError("Invalid rho parameter at index 1: {}".format(rho))
        if not 0.0 <= weight_decay:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))
        defaults = dict(lr=lr, betas=betas, rho=rho, 
                        weight_decay=weight_decay, 
                        maximize=maximize, capturable=capturable)
        super(SophiaG, self).__init__(params, defaults)



In [ ]:
#| export
@patch
def __setstate__(self: SophiaG, state):
        super().__setstate__(state)
        for group in self.param_groups:
            group.setdefault('maximize', False)
            group.setdefault('capturable', False)
        state_values = list(self.state.values())
        step_is_tensor = (len(state_values) != 0) and torch.is_tensor(state_values[0]['step'])
        if not step_is_tensor:
            for s in state_values:
                s['step'] = torch.tensor(float(s['step']))

In [ ]:
#| export
@patch
@torch.no_grad()
def update_hessian(self: SophiaG):
    for group in self.param_groups:
        beta1, beta2 = group['betas']
        for p in group['params']:
            if p.grad is None:
                continue
            state = self.state[p]

            if len(state) == 0:
                state['step'] = torch.zeros((1,), dtype=torch.float, device=p.device) \
                    if self.defaults['capturable'] else torch.tensor(0.)
                state['exp_avg'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                state['hessian'] = torch.zeros_like(p, memory_format=torch.preserve_format)
            
            if 'hessian' not in state.keys():
                state['hessian'] = torch.zeros_like(p, memory_format=torch.preserve_format)

            state['hessian'].mul_(beta2).addcmul_(p.grad, p.grad, value=1 - beta2)



In [ ]:
#| export
@patch
@torch.no_grad()
def step(self: SophiaG, closure=None, bs=5120):
    loss = None
    if closure is not None:
        with torch.enable_grad():
            loss = closure()

    for group in self.param_groups:
        params_with_grad = []
        grads = []
        exp_avgs = []
        state_steps = []
        hessian = []
        beta1, beta2 = group['betas']

        for p in group['params']:
            if p.grad is None:
                continue
            params_with_grad.append(p)
            
            if p.grad.is_sparse:
                raise RuntimeError('Hero does not support sparse gradients')
            grads.append(p.grad)
            state = self.state[p]
            # State initialization
            if len(state) == 0:
                state['step'] = torch.zeros((1,), dtype=torch.float, device=p.device) \
                    if self.defaults['capturable'] else torch.tensor(0.)
                state['exp_avg'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                state['hessian'] = torch.zeros_like(p, memory_format=torch.preserve_format)
            
            if 'hessian' not in state.keys():
                state['hessian'] = torch.zeros_like(p, memory_format=torch.preserve_format)                

            exp_avgs.append(state['exp_avg'])
            state_steps.append(state['step'])
            hessian.append(state['hessian'])
            
            if self.defaults['capturable']:
                bs = torch.ones((1,), dtype=torch.float, device=p.device) * bs

        sophiag(params_with_grad,
                grads,
                exp_avgs,
                hessian,
                state_steps,
                bs=bs,
                beta1=beta1,
                beta2=beta2,
                rho=group['rho'],
                lr=group['lr'],
                weight_decay=group['weight_decay'],
                maximize=group['maximize'],
                capturable=group['capturable'])

    return loss


In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()